In [9]:
import os
import numpy as np
import pandas as pd
from trackml.dataset import load_event
from trackml.randomize import shuffle_hits
from trackml.score import score_event
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import matplotlib.pyplot as plt
from hough import *
from conformalMap import *
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import layers
import tensorflow as tf

%matplotlib inline

### Lets split the truth data, and create a dataset with conformed points and target momentum

In [2]:
train_X = []
train_Y = []

for i in range(50):
    print(i)
    if i < 10:
        s = '0' + str(i)
    else:
        s = str(i)
    event_prefix = 'event0000010' + s
    hits, cells, particles, truth = load_event(os.path.join('train_100_events', event_prefix))
    averageNumberHits = particles['nhits'].mean()

    cond = (hits['volume_id'] == 8) | (hits['volume_id'] == 13) | (hits['volume_id'] == 17)
    selected_indices = hits.index[cond].tolist()
    selected_hits = hits.iloc[selected_indices]
    selected_truth = truth.iloc[selected_indices]


    tracks = selected_truth.particle_id.unique()
    selected_tracks = []
    for track in tracks:
        # A value of 0 means that the hit did not originate 
        # from a reconstructible particle, but e.g. from detector noise.
        if track == 0:
            continue
        t = selected_truth[selected_truth.particle_id == track]
        if len(t) == 8:
            selected_tracks.append(t)


    for i in range(len(selected_tracks)):
        train_X.append([])
        for x in zip(selected_tracks[i].tx, selected_tracks[i].ty):
            train_X[len(train_X) - 1].append(x)

        px = selected_tracks[i].tpx.mean()
        py = selected_tracks[i].tpy.mean()
        mom = np.sqrt(px**2 + py**2)
        train_Y.append(mom)
    
train_X = np.array(train_X)
train_Y = np.reshape(np.array(train_Y), (len(train_X), 1))
print("train_X shape:", train_X.shape)
print("train_Y shape:", train_Y.shape)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
train_X shape: (27188, 8, 2)
train_Y shape: (27188, 1)


## Perform conformal mapping on points:

In [3]:
conform_train_X = train_X
for i in range(len(conform_train_X)):
    p = conform_train_X[i]
    for j in range(len(p)):
        x = p[j][0]
        y = p[j][1]
        denom = (x**2 + y**2)
        newX = x / denom
        newY = y / denom
        p[j][0] = newX
        p[j][1] = newY
        
    conform_train_X[i] = p
print("Conformed train_X shape:", conform_train_X.shape)

Conformed train_X shape: (27188, 8, 2)


In [16]:
def build_model():
  model = keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=(8, 2)),
    layers.Flatten(),
    layers.Dense(1)
  ])

  #optimizer = keras.optimizers.RMSprop(learning_rate=0.00001) #0.00001 works good
  model.compile(loss='mse',
                optimizer='adam')
  return model

In [17]:
model = build_model()

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 8, 32)             96        
_________________________________________________________________
flatten_5 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 257       
Total params: 353
Trainable params: 353
Non-trainable params: 0
_________________________________________________________________


In [19]:
#example_batch = train_X[:1]
#example_result = model.predict(example_batch)
#example_result

In [20]:
EPOCHS = 200000
print(conform_train_X.shape)
print(train_Y.shape)
k = 1000
history = model.fit(conform_train_X[:k], train_Y[:k], epochs=EPOCHS, batch_size=32)

(27188, 8, 2)
(27188, 1)
Instructions for updating:
Use tf.cast instead.
Epoch 1/200000
1000/1000 [==============================] - 0s 209us/step - loss: 1.0827
Epoch 2/200000
1000/1000 [==============================] - 0s 22us/step - loss: 0.8899
Epoch 3/200000
1000/1000 [==============================] - 0s 23us/step - loss: 0.8713
Epoch 4/200000
1000/1000 [==============================] - 0s 23us/step - loss: 0.8718
Epoch 5/200000
1000/1000 [==============================] - 0s 23us/step - loss: 0.8734
Epoch 6/200000
1000/1000 [==============================] - 0s 25us/step - loss: 0.8715
Epoch 7/200000
1000/1000 [==============================] - 0s 22us/step - loss: 0.8720
Epoch 8/200000
1000/1000 [==============================] - 0s 25us/step - loss: 0.8719
Epoch 9/200000
1000/1000 [==============================] - 0s 25us/step - loss: 0.8718
Epoch 10/200000
1000/1000 [==============================] - 0s 31us/step - loss: 0.8720
Epoch 11/200000
1000/1000 [==================

1000/1000 [==============================] - 0s 23us/step - loss: 0.8711
Epoch 91/200000
1000/1000 [==============================] - 0s 27us/step - loss: 0.8705
Epoch 92/200000
1000/1000 [==============================] - 0s 24us/step - loss: 0.8713
Epoch 93/200000
1000/1000 [==============================] - 0s 25us/step - loss: 0.8710
Epoch 94/200000
1000/1000 [==============================] - 0s 23us/step - loss: 0.8716
Epoch 95/200000
1000/1000 [==============================] - 0s 22us/step - loss: 0.8741
Epoch 96/200000
1000/1000 [==============================] - 0s 22us/step - loss: 0.8707
Epoch 97/200000
1000/1000 [==============================] - 0s 22us/step - loss: 0.8712
Epoch 98/200000
1000/1000 [==============================] - 0s 22us/step - loss: 0.8732
Epoch 99/200000
1000/1000 [==============================] - 0s 22us/step - loss: 0.8711
Epoch 100/200000
1000/1000 [==============================] - 0s 22us/step - loss: 0.8706
Epoch 101/200000
1000/1000 [========

1000/1000 [==============================] - 0s 26us/step - loss: 0.8692
Epoch 182/200000
1000/1000 [==============================] - 0s 26us/step - loss: 0.8703
Epoch 183/200000
1000/1000 [==============================] - 0s 25us/step - loss: 0.8696
Epoch 184/200000
1000/1000 [==============================] - 0s 25us/step - loss: 0.8705
Epoch 185/200000
1000/1000 [==============================] - 0s 25us/step - loss: 0.8701
Epoch 186/200000
1000/1000 [==============================] - 0s 23us/step - loss: 0.8692
Epoch 187/200000
1000/1000 [==============================] - 0s 24us/step - loss: 0.8703
Epoch 188/200000
1000/1000 [==============================] - 0s 24us/step - loss: 0.8706
Epoch 189/200000
1000/1000 [==============================] - 0s 26us/step - loss: 0.8684
Epoch 190/200000
1000/1000 [==============================] - 0s 25us/step - loss: 0.8692
Epoch 191/200000
1000/1000 [==============================] - 0s 26us/step - loss: 0.8695
Epoch 192/200000
1000/1000 

1000/1000 [==============================] - 0s 25us/step - loss: 0.8696
Epoch 273/200000
1000/1000 [==============================] - 0s 26us/step - loss: 0.8684
Epoch 274/200000
1000/1000 [==============================] - 0s 26us/step - loss: 0.8675
Epoch 275/200000
1000/1000 [==============================] - 0s 25us/step - loss: 0.8674
Epoch 276/200000
1000/1000 [==============================] - 0s 24us/step - loss: 0.8686
Epoch 277/200000
1000/1000 [==============================] - 0s 24us/step - loss: 0.8683
Epoch 278/200000
1000/1000 [==============================] - 0s 25us/step - loss: 0.8672
Epoch 279/200000
1000/1000 [==============================] - 0s 24us/step - loss: 0.8681
Epoch 280/200000
1000/1000 [==============================] - 0s 24us/step - loss: 0.8687
Epoch 281/200000
1000/1000 [==============================] - 0s 24us/step - loss: 0.8676
Epoch 282/200000
1000/1000 [==============================] - 0s 26us/step - loss: 0.8668
Epoch 283/200000
1000/1000 

1000/1000 [==============================] - 0s 26us/step - loss: 0.8658
Epoch 364/200000
1000/1000 [==============================] - 0s 26us/step - loss: 0.8651
Epoch 365/200000
1000/1000 [==============================] - 0s 29us/step - loss: 0.8649
Epoch 366/200000
1000/1000 [==============================] - 0s 30us/step - loss: 0.8653
Epoch 367/200000
1000/1000 [==============================] - 0s 33us/step - loss: 0.8651
Epoch 368/200000
1000/1000 [==============================] - 0s 33us/step - loss: 0.8668
Epoch 369/200000
1000/1000 [==============================] - 0s 35us/step - loss: 0.8649
Epoch 370/200000
1000/1000 [==============================] - 0s 36us/step - loss: 0.8646
Epoch 371/200000
1000/1000 [==============================] - 0s 38us/step - loss: 0.8673
Epoch 372/200000
1000/1000 [==============================] - 0s 38us/step - loss: 0.8643
Epoch 373/200000
1000/1000 [==============================] - 0s 41us/step - loss: 0.8650
Epoch 374/200000
1000/1000 

...

1000/1000 [==============================] - 0s 26us/step - loss: 0.8209
Epoch 5231/200000
1000/1000 [==============================] - 0s 25us/step - loss: 0.8208
Epoch 5232/200000
1000/1000 [==============================] - 0s 24us/step - loss: 0.8211
Epoch 5233/200000
1000/1000 [==============================] - 0s 24us/step - loss: 0.8231
Epoch 5234/200000
1000/1000 [==============================] - 0s 27us/step - loss: 0.8200
Epoch 5235/200000
1000/1000 [==============================] - 0s 26us/step - loss: 0.8232
Epoch 5236/200000
1000/1000 [==============================] - 0s 27us/step - loss: 0.8211
Epoch 5237/200000
1000/1000 [==============================] - 0s 26us/step - loss: 0.8200
Epoch 5238/200000
1000/1000 [==============================] - 0s 25us/step - loss: 0.8215
Epoch 5239/200000
1000/1000 [==============================] - 0s 31us/step - loss: 0.8204
Epoch 5240/200000
1000/1000 [==============================] - 0s 33us/step - loss: 0.8203
Epoch 5241/200000

1000/1000 [==============================] - 0s 26us/step - loss: 0.8198
Epoch 5321/200000
1000/1000 [==============================] - 0s 30us/step - loss: 0.8218
Epoch 5322/200000
1000/1000 [==============================] - 0s 26us/step - loss: 0.8189
Epoch 5323/200000
1000/1000 [==============================] - 0s 26us/step - loss: 0.8204
Epoch 5324/200000
1000/1000 [==============================] - 0s 26us/step - loss: 0.8198
Epoch 5325/200000
1000/1000 [==============================] - 0s 26us/step - loss: 0.8197
Epoch 5326/200000
1000/1000 [==============================] - 0s 27us/step - loss: 0.8184
Epoch 5327/200000
1000/1000 [==============================] - 0s 29us/step - loss: 0.8196
Epoch 5328/200000
1000/1000 [==============================] - 0s 27us/step - loss: 0.8205
Epoch 5329/200000
1000/1000 [==============================] - 0s 27us/step - loss: 0.8212
Epoch 5330/200000
1000/1000 [==============================] - 0s 27us/step - loss: 0.8182
Epoch 5331/200000

1000/1000 [==============================] - 0s 25us/step - loss: 0.8202
Epoch 5411/200000
1000/1000 [==============================] - 0s 28us/step - loss: 0.8198
Epoch 5412/200000
1000/1000 [==============================] - 0s 23us/step - loss: 0.8196
Epoch 5413/200000
1000/1000 [==============================] - 0s 23us/step - loss: 0.8186
Epoch 5414/200000
1000/1000 [==============================] - 0s 22us/step - loss: 0.8200
Epoch 5415/200000
1000/1000 [==============================] - 0s 22us/step - loss: 0.8204
Epoch 5416/200000
1000/1000 [==============================] - 0s 22us/step - loss: 0.8224
Epoch 5417/200000
1000/1000 [==============================] - 0s 23us/step - loss: 0.8216
Epoch 5418/200000
1000/1000 [==============================] - 0s 23us/step - loss: 0.8205
Epoch 5419/200000
1000/1000 [==============================] - 0s 23us/step - loss: 0.8188
Epoch 5420/200000
1000/1000 [==============================] - 0s 21us/step - loss: 0.8188
Epoch 5421/200000

1000/1000 [==============================] - 0s 24us/step - loss: 0.8186
Epoch 5501/200000
1000/1000 [==============================] - 0s 23us/step - loss: 0.8215
Epoch 5502/200000
1000/1000 [==============================] - 0s 22us/step - loss: 0.8172
Epoch 5503/200000
1000/1000 [==============================] - 0s 22us/step - loss: 0.8187
Epoch 5504/200000
1000/1000 [==============================] - 0s 23us/step - loss: 0.8211
Epoch 5505/200000
1000/1000 [==============================] - 0s 22us/step - loss: 0.8213
Epoch 5506/200000
1000/1000 [==============================] - 0s 22us/step - loss: 0.8192
Epoch 5507/200000
  32/1000 [..............................] - ETA: 0s - loss: 0.1434

KeyboardInterrupt: 

In [ ]:
prediction = model.predict(conform_train_X[:k])
print(prediction)
print(train_Y[:k])
print(train_Y.mean())